# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=31

In [3]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==31]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm31', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm31/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale
2017-08-29 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25938/2831038004.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25938/2831038004.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25938/2831038004.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]         [,4]
sigmas -0.916643775 1.2395411 -1.436006819   0.01996803
       -0.906363699 1.2410565 -1.492993244   0.32885632
       -0.896188229 1.2421357 -1.547720994   0.63284897
       -0.886115256 1.2427923 -1.600225823   0.93198617
       -0.876142738 1.2430396 -1.650542894   1.22630680
       -0.866268689 1.2428907 -1.698706792   1.51584839
       -0.856491185 1.2423585 -1.744751542   1.80064718
       -0.846808355 1.2414555 -1.788710615   2.08073811
       -0.837218384 1.2401940 -1.830616949   2.35615479
       -0.827719508 1.2385863 -1.870502958   2.62692955
       -0.818310012 1.2366442 -1.908400545   2.89309341
       -0.808988229 1.2343794 -1.944341118   3.15467610
       -0.799752541 1.2318034 -1.978355602   3.41170608
       -0.790601371 1.2289273 -2.010474451   3.66421052
       -0.781533185 1.2257623 -2.040727662   3.91221532
       -0.77254

       -0.009340127 0.7393133  0.473164577  -2.19748686
       -0.005178247 0.7412898  0.467064856  -2.54515374
       -0.001033617 0.7432501  0.459392352  -2.89627833
        0.003093906 0.7451868  0.450127915  -3.25078978
        0.007204463 0.7470928  0.439252987  -3.60861630
        0.011298192 0.7489608  0.426749617  -3.96968515
        0.015375230 0.7507832  0.412600459  -4.33392272
        0.019435714 0.7525526  0.396788789  -4.70125454
        0.023479777 0.7542614  0.379298507  -5.07160539
        0.027507552 0.7559020  0.360114142  -5.44489927
        0.031519168 0.7574665  0.339220862  -5.82105952
        0.035514756 0.7589472  0.316604476  -6.20000880
        0.039494443 0.7603361  0.292251438  -6.58166919
        0.043458354 0.7616255  0.266148854  -6.96596219
        0.047406615 0.7628073  0.238284483  -7.35280883
        0.051339348 0.7638734  0.208646741  -7.74212963
        0.055256676 0.7648159  0.177224700  -8.13384471
        0.059158718 0.7656266  0.144008097  -8.5

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25938/2831038004.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25938/2831038004.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25938/2831038004.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

...

[1] 44.694713  4.722982 66.258718
                [,1]      [,2]          [,3]         [,4]
sigmas -0.6946707214 1.2246851 -1.642455e+00 1.7523733057
       -0.6828124351 1.2150776 -1.633885e+00 1.7748073183
       -0.6710931215 1.2055110 -1.625111e+00 1.7968470280
       -0.6595095607 1.1959868 -1.616138e+00 1.8184952991
       -0.6480586438 1.1865067 -1.606969e+00 1.8397550060
       -0.6367373674 1.1770721 -1.597609e+00 1.8606290311
       -0.6255428287 1.1676847 -1.588061e+00 1.8811202607
       -0.6144722216 1.1583459 -1.578330e+00 1.9012315811
       -0.6035228322 1.1490571 -1.568417e+00 1.9209658761
       -0.5926920345 1.1398199 -1.558328e+00 1.9403260234
       -0.5819772872 1.1306355 -1.548066e+00 1.9593148916
       -0.5713761298 1.1215054 -1.537634e+00 1.9779353373
       -0.5608861789 1.1124309 -1.527036e+00 1.9961902020
       -0.5505051258 1.1034133 -1.516275e+00 2.0140823099
       -0.5402307327 1.0944538 -1.505355e+00 2.0316144647
       -0.5300608301 1.0855538 -1.4942

        0.2829459582 0.6223077  2.266511e-01 1.8640196051
        0.2874236403 0.6237374  2.372090e-01 1.8439184281
        0.2918813622 0.6252281  2.476106e-01 1.8234745136
        0.2963193009 0.6267786  2.578530e-01 1.8026861419
        0.3007376313 0.6283879  2.679334e-01 1.7815515949
        0.3051365259 0.6300546  2.778486e-01 1.7600691574
        0.3095161549 0.6317778  2.875959e-01 1.7382371182
        0.3138766864 0.6335562  2.971721e-01 1.7160537716
        0.3182182862 0.6353886  3.065742e-01 1.6935174188
        0.3225411179 0.6372737  3.157994e-01 1.6706263693
        0.3268453432 0.6392104  3.248444e-01 1.6473789423
        0.3311311215 0.6411973  3.337063e-01 1.6237734681
        0.3353986102 0.6432332  3.423820e-01 1.5998082896
        0.3396479649 0.6453166  3.508683e-01 1.5754817639
        0.3438793389 0.6474463  3.591622e-01 1.5507922632
        0.3480928838 0.6496209  3.672605e-01 1.5257381771
        0.3522887492 0.6518389  3.751602e-01 1.5003179136
        0.3564

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25938/2831038004.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25938/2831038004.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25938/2831038004.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]        [,3]        [,4]
sigmas -0.900527607 1.4539965 -2.84542173  5.38989808
       -0.878050546 1.4287055 -2.80077069  5.37188297
       -0.856067616 1.4037799 -2.75608968  5.35312646
       -0.834557560 1.3792218 -2.71138982  5.33363395
       -0.813500459 1.3550331 -2.66668222  5.31341117
       -0.792877632 1.3312157 -2.62197797  5.29246419
       -0.772671526 1.3077712 -2.57728815  5.27079936
       -0.752865633 1.2847012 -2.53262379  5.24842336
       -0.733444407 1.2620072 -2.48799586  5.22534312
       -0.714393191 1.2396903 -2.44341532  5.20156590
       -0.695698149 1.2177517 -2.39889303  5.17709920
       -0.677346206 1.1961924 -2.35443980  5.15195080
       -0.659324997 1.1750132 -2.31006637  5.12612872
       -0.641622812 1.1542149 -2.26578337  5.09964121
       -0.624228550 1.1337981 -2.22160136  5.07249678
       -0.607131684 1.1137632 -2.17753080  5.04470414
       -0.590322213 1.0941106 -2.13358204  5.016

        0.583287235 0.8521592  0.53306551 -0.75109380
        0.588428855 0.8568609  0.52689396 -0.78605371
        0.593544173 0.8615067  0.52046440 -0.82064587
        0.598633459 0.8660948  0.51378230 -0.85486690
        0.603696974 0.8706233  0.50685316 -0.88871355
        0.608734979 0.8750908  0.49968253 -0.92218267
        0.613747730 0.8794954  0.49227597 -0.95527125
        0.618735479 0.8838358  0.48463909 -0.98797636
        0.623698473 0.8881104  0.47677752 -1.02029523
        0.628636958 0.8923179  0.46869692 -1.05222520
        0.633551174 0.8964570  0.46040295 -1.08376374
        0.638441358 0.9005264  0.45190132 -1.11490844
        0.643307745 0.9045250  0.44319772 -1.14565704
        0.648150565 0.9084516  0.43429788 -1.17600742
        0.652970045 0.9123053  0.42520752 -1.20595758
        0.657766408 0.9160851  0.41593236 -1.23550568
        0.662539877 0.9197902  0.40647813 -1.26465000
        0.667290668 0.9234197  0.39685056 -1.29338900
        0.672018995 0.926972

In [4]:
print(error_i)

[2, 4, 6, 10, 11, 47]
